In [1]:
from __future__ import print_function

When analyzing data, I usually use the following three modules. I use pandas for data management, filtering, grouping, and processing. I use numpy for basic array math. I use toyplot for rendering the charts.

In [2]:
import pandas
import numpy
import toyplot
import toyplot.pdf
import toyplot.png
import toyplot.svg

print('Pandas version:  ', pandas.__version__)
print('Numpy version:   ', numpy.__version__)
print('Toyplot version: ', toyplot.__version__)

Pandas version:   0.19.2
Numpy version:    1.12.0
Toyplot version:  0.14.0-dev


Load in the "auto" dataset. This is a fun collection of data on cars manufactured between 1970 and 1982. The source for this data can be found at https://archive.ics.uci.edu/ml/datasets/Auto+MPG.

The data are stored in a text file containing columns of data. We use the pandas.read_table() method to parse the data and load it in a pandas DataFrame. The file does not contain a header row, so we need to specify the names of the columns manually.

In [3]:
column_names = ['MPG',
                'Cylinders',
                'Displacement',
                'Horsepower',
                'Weight',
                'Acceleration',
                'Model Year',
                'Origin',
                'Car Name']
data = pandas.read_table('auto-mpg.data',
                         delim_whitespace=True,
                         names=column_names,
                         index_col=False)

The origin column indicates the country of origin for the car manufacture. It has three numeric values, 1, 2, or 3. These indicate USA, Europe, or Japan, respectively. Replace the origin column with a string representing the country name.

In [4]:
country_map = pandas.Series(index=[1,2,3],
                            data=['USA', 'Europe', 'Japan'])
data['Origin'] = numpy.array(country_map[data['Origin']])

In this plot we are going to show the trend of the average miles per gallon (MPG) rating for subsequent model years separated by country of origin. This time period saw a significant increase in MPG driven by the U.S. fuel crisis. We can use the pivot_table feature of pandas to get this information from the data. (Excel and other spreadsheets have similar functionality.)

In [5]:
average_mpg_per_year = data.pivot_table(index='Model Year',
                                        columns='Origin',
                                        values='MPG',
                                        aggfunc='mean')
average_mpg_per_year

Origin,Europe,Japan,USA
Model Year,,,
70,25.200000,25.500000,15.272727
71,28.750000,29.500000,18.100000
72,22.000000,24.200000,16.277778
73,24.000000,20.000000,15.034483
74,27.000000,29.333333,18.333333
75,24.500000,27.500000,17.550000
76,24.250000,28.000000,19.431818
77,29.250000,27.416667,20.722222
78,24.950000,29.687500,21.772727


In [6]:
average_mpg_per_year.columns

Index([u'Europe', u'Japan', u'USA'], dtype='object', name=u'Origin')

Now use toyplot to plot this trend on a standard x-y chart.

In [7]:
canvas = toyplot.Canvas('4in', '2.6in')

axes = canvas.cartesian(bounds=(41,-1,6,-43),
                        xlabel = 'Model Year',
                        ylabel = 'Average MPG')

for column in country_map:
    series = average_mpg_per_year[column]
    x = series.index + 1900
    y = numpy.array(series)
    axes.plot(x, y)
    axes.text(x[-1], y[-1], column,
              style={"text-anchor":"start",
                     "-toyplot-anchor-shift":"2px"})

# It's usually best to make the y-axis 0-based.
axes.y.domain.min = 0

# Toyplot is sometimes inaccurate in judging the width of labels.
axes.x.domain.max = 1984

# The labels can make for odd tick placement.
# Place them manually
axes.x.ticks.locator = \
    toyplot.locator.Explicit([1970,1974,1978,1982])

<svg class="toyplot-canvas-Canvas" height="249.60000000000002px" id="t39ae15aafa02411881f380397ee8edfa" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 384.0 249.60000000000002" width="384.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 41.0 131.41091264995552 L 65.428571428571416 117.4919804083699 L 89.857142857142861 126.46295352870588 L 114.28571428571428 132.58381302733716 L 138.71428571428569 116.34325824052199 L 163.14285714285714 120.19968266115423 L 187.57142857142856 110.93531296980944 L 212.0 104.58253128398395 L 236.42857142857142 99.410795117049673 L 260.85714285714289 91.014291185490592 L 285.28571428571428 79.021509440239114 L 309.71428571428572 71.06340751447334 L 334.14285714285711 61.614852100911222" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> USA <path d="M 41.0 82.538005872426581 L 65.428571428571416 65.061018604454915 L 89.857142857142861 98.291909888626392 L 114.28571428571428 88.445719878501507 L 138.71428571428569 73.676434863314199 L 163.14285714285714 85.984172375970275 L 187.57142857142856 87.214946127235905 L 212.0 62.599471101923697 L 236.42857142857142 83.768779623692211 L 260.85714285714289 56.691757095848779 L 285.28571428571428 23.023257366782879 L 309.71428571428572 51.153275215153528 L 334.14285714285711 9.6761997975024894" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Europe <path d="M 41.0 81.061077370907839 L 65.428571428571416 61.368697350658096 L 89.857142857142861 87.461100877489031 L 114.28571428571428 108.13809989875126 L 138.71428571428569 62.189213184835175 L 163.14285714285714 71.214887360782967 L 187.57142857142856 68.753339858251763 L 212.0 71.625145277871511 L 236.42857142857142 60.445617037208905 L 260.85714285714289 44.384019583192682 L 285.28571428571428 32.322436820789726 L 309.71428571428572 44.342993791483863 L 334.14285714285711 34.838685378932745" style="stroke:rgb(55.3%,62.7%,79.6%);stroke-opacity:1.0;stroke-width:2.0"> Japan 1970 1974 1978 1982 Model Year 0 10 20 30 40 Average MPG 
 
 
 Save as .csv

In [8]:
toyplot.pdf.render(canvas, 'MultiSeries.pdf')
toyplot.svg.render(canvas, 'MultiSeries.svg')
toyplot.png.render(canvas, 'MultiSeries.png', scale=5)

For the talk, I want to compare this to using a 3D plot. Toyplot does not yet have such silly plot capabilities, so write out the results of this pivot table to csv so we can easily load it into Excel. 

In [9]:
average_mpg_per_year.to_csv('auto-mpg-origin-year.csv')

In one of my counterexamples, I remind the audiance to make colors consistent. Make a plot with inconsistent colors.

In [10]:
canvas = toyplot.Canvas('4in', '2.6in')

axes = canvas.cartesian(bounds=(41,-1,6,-43),
                        xlabel = 'Model Year',
                        ylabel = 'Average MPG')

for column in ['Europe', 'Japan', 'USA']:
    series = average_mpg_per_year[column]
    x = series.index + 1900
    y = numpy.array(series)
    axes.plot(x, y)
    axes.text(x[-1], y[-1], column,
              style={"text-anchor":"start",
                     "-toyplot-anchor-shift":"2px"})

# It's usually best to make the y-axis 0-based.
axes.y.domain.min = 0

# Toyplot is sometimes inaccurate in judging the width of labels.
axes.x.domain.max = 1984

# The labels can make for odd tick placement.
# Place them manually
axes.x.ticks.locator = \
    toyplot.locator.Explicit([1970,1974,1978,1982])

<svg class="toyplot-canvas-Canvas" height="249.60000000000002px" id="t805210c2086f458194e15b0db0148665" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 384.0 249.60000000000002" width="384.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 41.0 82.538005872426581 L 65.428571428571416 65.061018604454915 L 89.857142857142861 98.291909888626392 L 114.28571428571428 88.445719878501507 L 138.71428571428569 73.676434863314199 L 163.14285714285714 85.984172375970275 L 187.57142857142856 87.214946127235905 L 212.0 62.599471101923697 L 236.42857142857142 83.768779623692211 L 260.85714285714289 56.691757095848779 L 285.28571428571428 23.023257366782879 L 309.71428571428572 51.153275215153528 L 334.14285714285711 9.6761997975024894" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> Europe <path d="M 41.0 81.061077370907839 L 65.428571428571416 61.368697350658096 L 89.857142857142861 87.461100877489031 L 114.28571428571428 108.13809989875126 L 138.71428571428569 62.189213184835175 L 163.14285714285714 71.214887360782967 L 187.57142857142856 68.753339858251763 L 212.0 71.625145277871511 L 236.42857142857142 60.445617037208905 L 260.85714285714289 44.384019583192682 L 285.28571428571428 32.322436820789726 L 309.71428571428572 44.342993791483863 L 334.14285714285711 34.838685378932745" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Japan <path d="M 41.0 131.41091264995552 L 65.428571428571416 117.4919804083699 L 89.857142857142861 126.46295352870588 L 114.28571428571428 132.58381302733716 L 138.71428571428569 116.34325824052199 L 163.14285714285714 120.19968266115423 L 187.57142857142856 110.93531296980944 L 212.0 104.58253128398395 L 236.42857142857142 99.410795117049673 L 260.85714285714289 91.014291185490592 L 285.28571428571428 79.021509440239114 L 309.71428571428572 71.06340751447334 L 334.14285714285711 61.614852100911222" style="stroke:rgb(55.3%,62.7%,79.6%);stroke-opacity:1.0;stroke-width:2.0"> USA 1970 1974 1978 1982 Model Year 0 10 20 30 40 Average MPG 
 
 
 Save as .csv

In [11]:
toyplot.pdf.render(canvas, 'MultiSeries_Inconsistent_Colors.pdf')
toyplot.svg.render(canvas, 'MultiSeries_Inconsistent_colors.svg')
toyplot.png.render(canvas, 'MultiSeries_Inconsistent_colors.png', scale=5)

I make a point that it is a bad idea to clutter up the canvas with non-data items like grid lines. Create a counter example that has lots of distracting lines.

In [12]:
canvas = toyplot.Canvas('4in', '2.6in')

axes = canvas.cartesian(bounds=(41,-1,6,-43),
                        xlabel = 'Model Year',
                        ylabel = 'Average MPG')
    
# Create some grid lines. (Not a great idea.)
axes.hlines(xrange(0,41,5), color='black')
axes.vlines(xrange(1970,1983), color='black')

for column in country_map:
    series = average_mpg_per_year[column]
    x = series.index + 1900
    y = numpy.array(series)
    axes.plot(x, y)
    axes.text(x[-1], y[-1], column,
              style={"text-anchor":"start",
                     "-toyplot-anchor-shift":"2px"})

# It's usually best to make the y-axis 0-based.
axes.y.domain.min = 0

# Toyplot is sometimes inaccurate in judging the width of labels.
axes.x.domain.max = 1984

# The labels can make for odd tick placement.
# Place them manually
axes.x.ticks.locator = \
    toyplot.locator.Explicit([1970,1974,1978,1982])

<svg class="toyplot-canvas-Canvas" height="249.60000000000002px" id="td2b342d400534e068ab1768b99ab0be8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 384.0 249.60000000000002" width="384.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 41.0 132.23165009240518 L 65.428571428571416 118.46465198451112 L 89.857142857142861 127.33770114015273 L 114.28571428571428 133.39174750475681 L 138.71428571428569 117.32846886092288 L 163.14285714285714 121.14279791868344 L 187.57142857142856 111.97955480506907 L 212.0 105.69611783370981 L 236.42857142857142 100.58083450673236 L 260.85714285714289 92.275983837703606 L 285.28571428571428 80.414111049647332 L 309.71428571428572 72.542876945416609 L 334.14285714285711 63.197458615682464" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> USA <path d="M 41.0 83.892222652468547 L 65.428571428571416 66.606007986447224 L 89.857142857142861 99.474162633107454 L 114.28571428571428 89.735450145208119 L 138.71428571428569 75.127381413359132 L 163.14285714285714 87.300772023233307 L 187.57142857142856 88.518111084220706 L 212.0 64.171329864472398 L 236.42857142857142 85.109561713455946 L 260.85714285714289 58.328102371732804 L 285.28571428571428 25.027116058943701 L 309.71428571428572 52.850076597289444 L 334.14285714285711 11.825750242013532" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Europe <path d="M 41.0 82.431415779283626 L 65.428571428571416 62.953990803484984 L 89.857142857142861 88.761578896418186 L 114.28571428571428 109.21287512100677 L 138.71428571428569 63.765550177476591 L 163.14285714285714 72.692703291384305 L 187.57142857142856 70.258025169409464 L 212.0 73.098482978380105 L 236.42857142857142 62.040986507744442 L 260.85714285714289 46.154711761858636 L 285.28571428571428 34.224788964181975 L 309.71428571428572 46.114133793159091 L 334.14285714285711 36.713571044422928" style="stroke:rgb(55.3%,62.7%,79.6%);stroke-opacity:1.0;stroke-width:2.0"> Japan 1970 1974 1978 1982 Model Year 0 10 20 30 40 Average MPG 
 
 
 Save as .csv

In [13]:
toyplot.pdf.render(canvas, 'MultiSeries_Grid_Dark.pdf')
toyplot.svg.render(canvas, 'MultiSeries_Grid_Dark.svg')
toyplot.png.render(canvas, 'MultiSeries_Grid_Dark.png', scale=5)

If you really want gridlines, you should make them very subtle so they don't interfere with the actual data.

In [14]:
canvas = toyplot.Canvas('4in', '2.6in')

axes = canvas.cartesian(bounds=(41,-1,6,-43),
                        xlabel = 'Model Year',
                        ylabel = 'Average MPG')
    
# Create some grid lines. (Not a great idea.)
axes.hlines(xrange(0,41,5), color='lightgray')
axes.vlines(xrange(1970,1983), color='lightgray')

for column in country_map:
    series = average_mpg_per_year[column]
    x = series.index + 1900
    y = numpy.array(series)
    axes.plot(x, y)
    axes.text(x[-1], y[-1], column,
              style={"text-anchor":"start",
                     "-toyplot-anchor-shift":"2px"})

# It's usually best to make the y-axis 0-based.
axes.y.domain.min = 0

axes.x.domain.max = 1984

# The labels can make for odd tick placement.
# Place them manually
axes.x.ticks.locator = \
    toyplot.locator.Explicit([1970,1974,1978,1982])

<svg class="toyplot-canvas-Canvas" height="249.60000000000002px" id="t31f03e7b445046ebba7cc9a2167af2ce" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 384.0 249.60000000000002" width="384.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 41.0 132.23165009240518 L 65.428571428571416 118.46465198451112 L 89.857142857142861 127.33770114015273 L 114.28571428571428 133.39174750475681 L 138.71428571428569 117.32846886092288 L 163.14285714285714 121.14279791868344 L 187.57142857142856 111.97955480506907 L 212.0 105.69611783370981 L 236.42857142857142 100.58083450673236 L 260.85714285714289 92.275983837703606 L 285.28571428571428 80.414111049647332 L 309.71428571428572 72.542876945416609 L 334.14285714285711 63.197458615682464" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> USA <path d="M 41.0 83.892222652468547 L 65.428571428571416 66.606007986447224 L 89.857142857142861 99.474162633107454 L 114.28571428571428 89.735450145208119 L 138.71428571428569 75.127381413359132 L 163.14285714285714 87.300772023233307 L 187.57142857142856 88.518111084220706 L 212.0 64.171329864472398 L 236.42857142857142 85.109561713455946 L 260.85714285714289 58.328102371732804 L 285.28571428571428 25.027116058943701 L 309.71428571428572 52.850076597289444 L 334.14285714285711 11.825750242013532" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Europe <path d="M 41.0 82.431415779283626 L 65.428571428571416 62.953990803484984 L 89.857142857142861 88.761578896418186 L 114.28571428571428 109.21287512100677 L 138.71428571428569 63.765550177476591 L 163.14285714285714 72.692703291384305 L 187.57142857142856 70.258025169409464 L 212.0 73.098482978380105 L 236.42857142857142 62.040986507744442 L 260.85714285714289 46.154711761858636 L 285.28571428571428 34.224788964181975 L 309.71428571428572 46.114133793159091 L 334.14285714285711 36.713571044422928" style="stroke:rgb(55.3%,62.7%,79.6%);stroke-opacity:1.0;stroke-width:2.0"> Japan 1970 1974 1978 1982 Model Year 0 10 20 30 40 Average MPG 
 
 
 Save as .csv

In [15]:
toyplot.pdf.render(canvas, 'MultiSeries_Grid_Light.pdf')
toyplot.svg.render(canvas, 'MultiSeries_Grid_Light.svg')
toyplot.png.render(canvas, 'MultiSeries_Grid_Light.png', scale=5)

Frankly, vertical gridlines are usually not all that necessary. If you remove them, less clutter. Not going overboard on horizontal lines is also good.

In [16]:
canvas = toyplot.Canvas('4in', '2.6in')

axes = canvas.cartesian(bounds=(41,-1,6,-43),
                        xlabel = 'Model Year',
                        ylabel = 'Average MPG')
    
# Create some grid lines. (Not a great idea.)
axes.hlines(xrange(0,41,10), color='lightgray')

for column in country_map:
    series = average_mpg_per_year[column]
    x = series.index + 1900
    y = numpy.array(series)
    axes.plot(x, y)
    axes.text(x[-1], y[-1], column,
              style={"text-anchor":"start",
                     "-toyplot-anchor-shift":"2px"})

# It's usually best to make the y-axis 0-based.
axes.y.domain.min = 0

axes.x.domain.max = 1984

# The labels can make for odd tick placement.
# Place them manually
axes.x.ticks.locator = \
    toyplot.locator.Explicit([1970,1974,1978,1982])

<svg class="toyplot-canvas-Canvas" height="249.60000000000002px" id="td320dd9a318d49e28b8c018285461a75" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 384.0 249.60000000000002" width="384.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 41.0 132.23165009240518 L 65.428571428571416 118.46465198451112 L 89.857142857142861 127.33770114015273 L 114.28571428571428 133.39174750475681 L 138.71428571428569 117.32846886092288 L 163.14285714285714 121.14279791868344 L 187.57142857142856 111.97955480506907 L 212.0 105.69611783370981 L 236.42857142857142 100.58083450673236 L 260.85714285714289 92.275983837703606 L 285.28571428571428 80.414111049647332 L 309.71428571428572 72.542876945416609 L 334.14285714285711 63.197458615682464" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> USA <path d="M 41.0 83.892222652468547 L 65.428571428571416 66.606007986447224 L 89.857142857142861 99.474162633107454 L 114.28571428571428 89.735450145208119 L 138.71428571428569 75.127381413359132 L 163.14285714285714 87.300772023233307 L 187.57142857142856 88.518111084220706 L 212.0 64.171329864472398 L 236.42857142857142 85.109561713455946 L 260.85714285714289 58.328102371732804 L 285.28571428571428 25.027116058943701 L 309.71428571428572 52.850076597289444 L 334.14285714285711 11.825750242013532" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Europe <path d="M 41.0 82.431415779283626 L 65.428571428571416 62.953990803484984 L 89.857142857142861 88.761578896418186 L 114.28571428571428 109.21287512100677 L 138.71428571428569 63.765550177476591 L 163.14285714285714 72.692703291384305 L 187.57142857142856 70.258025169409464 L 212.0 73.098482978380105 L 236.42857142857142 62.040986507744442 L 260.85714285714289 46.154711761858636 L 285.28571428571428 34.224788964181975 L 309.71428571428572 46.114133793159091 L 334.14285714285711 36.713571044422928" style="stroke:rgb(55.3%,62.7%,79.6%);stroke-opacity:1.0;stroke-width:2.0"> Japan 1970 1974 1978 1982 Model Year 0 10 20 30 40 Average MPG 
 
 
 Save as .csv

In [17]:
toyplot.pdf.render(canvas, 'MultiSeries_Grid_Light_Fewer.pdf')
toyplot.svg.render(canvas, 'MultiSeries_Grid_Light_Fewer.svg')
toyplot.png.render(canvas, 'MultiSeries_Grid_Light_Fewer.png', scale=5)

I personally find grid lines a bit overrated. Don't fear not having grid lines at all, as in the first example.

Another pet peeve of my is legends. I hate them. They are stupid and only exist because those who make plots are too lazy to place labels well (and because that is hard). But if you use a legend, at least make sure the order in the legend is not inconsistent with the order of the data.

In [18]:
canvas = toyplot.Canvas('4in', '2.6in')

axes = canvas.cartesian(bounds=(41,-11,6,-43),
                        xlabel = 'Model Year',
                        ylabel = 'Average MPG')

marks = {}
for column in country_map:
    series = average_mpg_per_year[column]
    x = series.index + 1900
    y = numpy.array(series)
    marks[column] = axes.plot(x, y)

# It's usually best to make the y-axis 0-based.
axes.y.domain.min = 0

# The labels can make for odd tick placement.
# Place them manually
axes.x.ticks.locator = \
    toyplot.locator.Explicit([1970,1974,1978,1982])
    
canvas.legend([('USA', marks['USA']),
               ('Europe', marks['Europe']),
               ('Japan', marks['Japan'])],
              rect=('-1in', '-1.25in', '1in', '0.75in'))

<svg class="toyplot-canvas-Canvas" height="249.60000000000002px" id="t5c5f333f834b4f0ba2452bfa5a3fa434" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 384.0 249.60000000000002" width="384.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 41.0 130.00727272727275 L 68.666666666666657 115.82850000000002 L 96.333333333333343 124.96694444444445 L 124.0 131.20206896551727 L 151.66666666666666 114.65833333333336 L 179.33333333333334 118.58675000000002 L 207.0 109.14943181818182 L 234.66666666666669 102.67805555555557 L 262.33333333333331 97.409772727272738 L 290.0 88.856521739130429 L 317.66666666666669 76.639857142857139 L 345.33333333333331 68.533192307692303 L 373.0 58.90825000000001" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 41.0 80.222000000000008 L 68.666666666666657 62.418750000000003 L 96.333333333333343 96.269999999999996 L 124.0 86.240000000000009 L 151.66666666666666 71.194999999999993 L 179.33333333333334 83.732500000000002 L 207.0 84.986250000000027 L 234.66666666666669 59.91125000000001 L 262.33333333333331 81.475750000000005 L 290.0 53.893250000000009 L 317.66666666666669 19.59622222222221 L 345.33333333333331 48.251375000000017 L 373.0 6.0" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 41.0 78.717500000000015 L 68.666666666666657 58.657499999999999 L 96.333333333333343 85.237000000000009 L 124.0 106.30000000000001 L 151.66666666666666 59.493333333333347 L 179.33333333333334 68.6875 L 207.0 66.180000000000021 L 234.66666666666669 69.10541666666667 L 262.33333333333331 57.71718750000003 L 290.0 41.355749999999986 L 317.66666666666669 29.069000000000003 L 345.33333333333331 41.31395833333336 L 373.0 31.632222222222246" style="stroke:rgb(55.3%,62.7%,79.6%);stroke-opacity:1.0;stroke-width:2.0"> 1970 1974 1978 1982 Model Year 0 10 20 30 40 Average MPG USA Europe Japan 
 
 
 Save as .csv

In [19]:
toyplot.pdf.render(canvas, 'Legend_Backward.pdf')
toyplot.svg.render(canvas, 'Legend_Backward.svg')
toyplot.png.render(canvas, 'Legend_Backward.png', scale=5)

Do it again, but at least order the legend correctly.

In [20]:
canvas = toyplot.Canvas('4in', '2.6in')

axes = canvas.cartesian(bounds=(41,-11,6,-43),
                        xlabel = 'Model Year',
                        ylabel = 'Average MPG')

marks = {}
for column in country_map:
    series = average_mpg_per_year[column]
    x = series.index + 1900
    y = numpy.array(series)
    marks[column] = axes.plot(x, y)

# It's usually best to make the y-axis 0-based.
axes.y.domain.min = 0

# The labels can make for odd tick placement.
# Place them manually
axes.x.ticks.locator = \
    toyplot.locator.Explicit([1970,1974,1978,1982])
    
canvas.legend([('Europe', marks['Europe']),
               ('Japan', marks['Japan']),
               ('USA', marks['USA'])],
              rect=('-1in', '-1.25in', '1in', '0.75in'))

<svg class="toyplot-canvas-Canvas" height="249.60000000000002px" id="t03ec14dd6d0548c492e318b4bcb8c8c2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 384.0 249.60000000000002" width="384.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 41.0 130.00727272727275 L 68.666666666666657 115.82850000000002 L 96.333333333333343 124.96694444444445 L 124.0 131.20206896551727 L 151.66666666666666 114.65833333333336 L 179.33333333333334 118.58675000000002 L 207.0 109.14943181818182 L 234.66666666666669 102.67805555555557 L 262.33333333333331 97.409772727272738 L 290.0 88.856521739130429 L 317.66666666666669 76.639857142857139 L 345.33333333333331 68.533192307692303 L 373.0 58.90825000000001" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 41.0 80.222000000000008 L 68.666666666666657 62.418750000000003 L 96.333333333333343 96.269999999999996 L 124.0 86.240000000000009 L 151.66666666666666 71.194999999999993 L 179.33333333333334 83.732500000000002 L 207.0 84.986250000000027 L 234.66666666666669 59.91125000000001 L 262.33333333333331 81.475750000000005 L 290.0 53.893250000000009 L 317.66666666666669 19.59622222222221 L 345.33333333333331 48.251375000000017 L 373.0 6.0" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 41.0 78.717500000000015 L 68.666666666666657 58.657499999999999 L 96.333333333333343 85.237000000000009 L 124.0 106.30000000000001 L 151.66666666666666 59.493333333333347 L 179.33333333333334 68.6875 L 207.0 66.180000000000021 L 234.66666666666669 69.10541666666667 L 262.33333333333331 57.71718750000003 L 290.0 41.355749999999986 L 317.66666666666669 29.069000000000003 L 345.33333333333331 41.31395833333336 L 373.0 31.632222222222246" style="stroke:rgb(55.3%,62.7%,79.6%);stroke-opacity:1.0;stroke-width:2.0"> 1970 1974 1978 1982 Model Year 0 10 20 30 40 Average MPG Europe Japan USA 
 
 
 Save as .csv

In [21]:
toyplot.pdf.render(canvas, 'Legend_OK.pdf')
toyplot.svg.render(canvas, 'Legend_OK.svg')
toyplot.png.render(canvas, 'Legend_OK.png', scale=5)